# 方策の混合を利用した解法？

気になるやり方を見つけたのでメモ

参考：
* [Reinforcement Learning with Convex Constraints](https://proceedings.neurips.cc/paper_files/paper/2019/hash/873be0705c80679f2c71fbf4d872df59-Abstract.html)
* [Batch Policy Learning under Constraints](https://proceedings.mlr.press/v97/le19a.html)
* [Distributionally Robust Constrained Reinforcement Learning under Strong Duality](https://arxiv.org/abs/2406.15788)


## [Reinforcement Learning with Convex Constraints](https://proceedings.neurips.cc/paper_files/paper/2019/hash/873be0705c80679f2c71fbf4d872df59-Abstract.html)

準備：
* MDP：$M=\left(\mathcal{S}, \mathcal{A}, \beta, P_s, P_z\right)$．ここで，$P_s$は繊維確率．$P_z$は$d$次元のベクトル$\mathbf{z}_i \in \mathbb{R}^d$を返す分布．$\mathbf{z}_i \sim P_z\left(\cdot \mid s_i, a_i\right)$みたいな感じ．

$$
\overline{\mathbf{z}}(\pi) \triangleq \mathbb{E}\left[\sum_{i=0}^{\infty} \gamma^i \mathbf{z}_i \mid \pi\right]
$$
とする．

---

**Mixed policyについて**

以降では有限個の方策の集合$\Pi$に対して，$\Pi$上の有限個の方策を選択し，それ上の分布$\mu$を考えることにします．
例えば$\Pi$はすべての決定的な方策の集合などですね．
つまり，$\mu \in \Delta (\Pi)$です．
$\mu$をデプロイする際は，あるエピソードにおいて，ランダムに$\mu$から$\pi$を選択し，そのエピソード中はずっとその$\pi$を使うことになります．

ここで，その$\mu$に従って得られる期待ベクトルを
$$
\overline{\mathbf{z}}(\mu) \triangleq \mathbb{E}_{\pi \sim \mu}[\overline{\mathbf{z}}(\pi)]=\sum_{\pi \in \Pi} \mu(\pi) \overline{\mathbf{z}}(\pi)
$$
と表記します．

---

以上の表記を使って，実行可能解を次のように定義します：

$$
\text { Feasibility Problem: Find } \mu \in \Delta(\Pi) \text { such that } \overline{\mathbf{z}}(\mu) \in \mathcal{C} \text {. }
$$

これを考えてもいいですが，最適化問題として定式化しましょう：

$$
\min _{\mu \in \Delta(\Pi)} \operatorname{dist}(\overline{\mathbf{z}}(\mu), \mathcal{C})
$$

つまり，$\overline{\mathbf{z}}(\mu)$と$\mathcal{C}$のユークリッド距離が最も近くなるような$\mu$を見つける問題ですね．

ここで，例えば凸錐$\mathcal{C}$に対して
$$
\mathcal{C}^{\circ} \triangleq\{\boldsymbol{\lambda}: \boldsymbol{\lambda} \cdot \mathrm{x} \leq 0 \text { for all } \mathrm{x} \in \mathcal{C}\}
$$
なるPolar Coneを考えると，次が成り立ちます（証明は論文のLemma 3.2）：
$$
\operatorname{dist}(\mathbf{x}, \mathcal{C})=\max _{\boldsymbol{\lambda} \in \mathcal{C}^{\circ} \cap \mathcal{B}} \boldsymbol{\lambda} \cdot \mathbf{x}
$$

こんな感じの表現を使えば，元の問題は
$$
\min _{\mu \in \Delta(\Pi)} \max _{\boldsymbol{\lambda} \in \Lambda} \boldsymbol{\lambda} \cdot \overline{\mathbf{z}}(\mu)
$$
と等価です．最適化関数が明らかに線形なので，強双対性が成り立ちます．

あとはBest-responseアルゴリズム（MDPで最適方策を返すやつ）を$\mu$側に適用し，とNo-regretアルゴリズム（sublinearリグレットを達成するオンラインアルゴリズム）を$\lambda$側に適用すれば解けます．


### Minimaxを解こう

ステップ $t=1, \dots, T$ について，学習者は$\lambda_t \in \Lambda$ の意思決定をし，環境は凸な損失 $\ell_t: \Lambda \to \mathbb{R}$について，$\ell_t(\lambda_t)$ を返す状況を考えます．
次のリグレットを定義しましょう：

$$
\operatorname{Regret}_T \triangleq\left[\sum_{t=1}^T \ell_t\left(\boldsymbol{\lambda}_t\right)\right]-\min _{\boldsymbol{\lambda} \in \Lambda}\left[\sum_{t=1}^T \ell_t(\boldsymbol{\lambda})\right]
$$

ここで，あるオンラインアルゴリズムは$\operatorname{Regret}_T=o(T)$のとき，**無後悔**であるといいます．例えばオンラインの勾配降下法などが無後悔アルゴリズムになります．

ここで，２プレイヤーのゼロサム・ゲームを考え，$\lambda \in \Lambda$と$u \in \mathcal{U}$のプレイヤーがいるとします．利得関数を$g(\lambda, u)$とします．
$g$が$\lambda$に凹で，$u$に凸であるとしましょう．$\Lambda$と$\mathcal{U}$が凸かつコンパクトなら，SionのMinimax定理から
$$
\max _{\boldsymbol{\lambda} \in \Lambda} \min _{\mathbf{u} \in \mathcal{U}} g(\boldsymbol{\lambda}, \mathbf{u})=\min _{\mathbf{u} \in \mathcal{U}} \max _{\boldsymbol{\lambda} \in \Lambda} g(\boldsymbol{\lambda}, \mathbf{u})
$$
が成り立ちます．

次のアルゴリズムを考えましょう．$t=1, \dots, T$について，
1. $\lambda_t$を無後悔アルゴリズムから吐き出します．
2. $u_t=\argmin_{u \in \mathcal{U}}g(\lambda_t, u)$と計算します．このような計算をbest-responseといいます．
3. 学習者は$\ell_t(\boldsymbol{\lambda})=-g\left(\boldsymbol{\lambda}, \mathbf{u}_t\right)$を観測します．

$\overline{\boldsymbol{\lambda}}=\frac{1}{T} \sum_{t=1}^T \boldsymbol{\lambda}_t$ および $\overline{\mathbf{u}}=\frac{1}{T} \sum_{t=1}^T \mathbf{u}_t$としましょう．
また，$\operatorname{Regret}_T$を3で定義された損失についてのリグレット関数とします．
つまり，
$$
\operatorname{Regret}_T \triangleq\max _{\boldsymbol{\lambda} \in \Lambda}\left[\sum_{t=1}^T g(\boldsymbol{\lambda}, u_t)\right] - \left[\sum_{t=1}^T g\left(\boldsymbol{\lambda}_t, u_t\right)\right]
$$

このとき，次が成立します．$v$をminimax問題の価値として，
$$
\min _{\mathbf{u} \in \mathcal{U}} g(\overline{\boldsymbol{\lambda}}, \mathbf{u}) \geq v-\delta \quad \text { and } \quad \max _{\boldsymbol{\lambda} \in \Lambda} g(\boldsymbol{\lambda}, \overline{\mathbf{u}}) \leq v+\delta, \quad \text { where } \delta=\frac{1}{T} \operatorname{Regret}_T .
$$

**証明**

まず，

$$
\begin{aligned}
\frac{1}{T} \sum_{t=1}^T g\left(\boldsymbol{\lambda}_t, \mathbf{u}_t\right) & =\frac{1}{T} \sum_{t=1}^T \min _{\mathbf{u} \in \mathcal{U}} g\left(\boldsymbol{\lambda}_t, \mathbf{u}\right) \\
& \leq \frac{1}{T} \min _{\mathbf{u} \in \mathcal{U}} \sum_{t=1}^T g\left(\boldsymbol{\lambda}_t, \mathbf{u}\right) \\
& \leq \min _{\mathbf{u} \in \mathcal{U}} g\left(\frac{1}{T} \sum_{t=1}^T \boldsymbol{\lambda}_t, \mathbf{u}\right) \\
& \leq \max _{\boldsymbol{\lambda} \in \Lambda} \min _{\mathbf{u} \in \mathcal{U}} g(\boldsymbol{\lambda}, \mathbf{u}) .
\end{aligned}
$$
が成り立ちます．ここで３行目は$g$の$\lambda$についての凹性のおかげです．$\delta=\frac{1}{T} \operatorname{Regret}_T$とすると，$\ell_t(\boldsymbol{\lambda})=-g\left(\boldsymbol{\lambda}, \mathbf{u}_t\right)$なので，

$$
\frac{1}{T} \sum_{t=1}^T g\left(\boldsymbol{\lambda}_t, \mathbf{u}_t\right)+\delta=\frac{1}{T} \max _{\boldsymbol{\lambda} \in \Lambda} \sum_{t=1}^T g\left(\boldsymbol{\lambda}, \mathbf{u}_t\right) \geq \max _{\boldsymbol{\lambda} \in \Lambda} g\left(\boldsymbol{\lambda}, \frac{1}{T} \sum_{t=1}^T \mathbf{u}_t\right) \geq \min _{\mathbf{u} \in \mathcal{U}} \max _{\boldsymbol{\lambda} \in \Lambda} g(\boldsymbol{\lambda}, \mathbf{u})
$$
が成り立ちます．ここで，１個目の不等式は$g$の$u$についての凸性を使ってます．

後は変形すれば終わりです．


### もとの問題に戻ろう

元々解きたかった問題に戻りましょう．
$
\overline{\mathbf{z}}(\mu) \triangleq \mathbb{E}_{\pi \sim \mu}[\overline{\mathbf{z}}(\pi)]=\sum_{\pi \in \Pi} \mu(\pi) \overline{\mathbf{z}}(\pi)
$
について，

$$
\min _{\mu \in \Delta(\Pi)} \max _{\boldsymbol{\lambda} \in \Lambda} \boldsymbol{\lambda} \cdot \overline{\mathbf{z}}(\mu)
$$
を解きたかったわけです．
ここで，$u_t=\mu_t$として，上のMinimaxの話に代入してみましょう．
$t=1, \dots, T$について，
1. $\lambda_t$を無後悔アルゴリズムから吐き出します．
2. $\mu_t=\argmin_{\mu \in \Delta(\Pi)}\lambda_t \cdot \overline{z}(\mu)$と計算します．このような計算をbest-responseといいます．
3. 学習者は$\ell_t(\boldsymbol{\lambda}_t)=-\lambda_t \cdot \overline{z}(\mu_t)$を観測します．

これを考えると，$\overline{\mu}=\frac{1}{T} \sum_{t=1}^T \mu_t$が最適な$\mu$に収束していきます．

**論文の13式あたりで次の式を使っているが，これはだめでは？**

$$
\overline{\mathbf{u}}=\frac{1}{T} \sum_{t=1}^T \overline{\mathbf{z}}\left(\pi_t\right)=\overline{\mathbf{z}}\left(\frac{1}{T} \sum_{t=1}^T \pi_t\right)
$$

例えば$z_i$が報酬だとすれば，$\overline{z}(\pi)$は割引期待収益を表します．
簡単のため$\pi_1$, $\pi_2$を考えましょう．上の式はつまり，
$$
\frac{\overline{z}(\pi_1) + \overline{z}(\pi_2)}{2}
= 
\overline{z}\left(\frac{\pi_1 + \pi_2}{2}\right)
$$
であることを主張してますが，これは[RL_value_function.ipynb](RL_value_function.ipynb)で使ったMDPの例を使えばわかりますが，成立しません．なのでこれはやっちゃだめです．

例えば$\mu_1$が$\pi_1$を確率１で出し，$\mu_2$が$\pi_2$を確率１で出すならば，$\overline{\mu}$は$\pi_1$，$\pi_2$をそれぞれ確率0.5ずつで出すだけで，その平均方策については何も言えません．
[Batch Policy Learning under Constraints](https://proceedings.mlr.press/v97/le19a.html)
はこの定式化に則ってるっぽいですが，かなり紛らわしいのでやめてほしい．


**注釈：この定式化はそもそもありなのか？**

* 「あるエピソードにおいて，ランダムに$\mu$から$\pi$を選択し，そのエピソード中はずっとその$\pi$を使うことになります．」
これをすると，$\Pi$が全部決定的な方策の場合に絶対に制約を満たさない方策が出てくる．
* また，$\mu$を使って得られるリターン
（$\overline{\mathbf{z}}(\mu) \triangleq \mathbb{E}_{\pi \sim \mu}[\overline{\mathbf{z}}(\pi)]=\sum \mu(\pi) \overline{\mathbf{z}}(\pi)$）
と，$\mu$を使って得られる方策が出すリターンって同じか？リターンは$\pi$について凸じゃないので，同じにはならないと思う．
  * 例えば$\Pi=\{\pi_1, \pi_2\}$で，$\mu$がその２つを等しく選択するとしよう．すると，$\pi_1$と$\pi_2$を別エピソードで選択する場合は$\overline{\mathbf{z}}(\mu) = 0.5 [\pi_1] + 0.5 [\pi_2]$になるが，$0.5 \pi_1 + 0.5 \pi_2$のリターンはそれと等しいわけではないはず（非凸なので）．
* 結論：これは方策のIdentificationには使えなさそう．
